# Denoising Autoencoder

In [1]:
import NBInclude: @nbinclude
@nbinclude("../Helpers/Base.ipynb");

In [2]:
# Parameters
model = "item_based_collaborative_filtering" #
content = "explicit" #
input_alphas = ["ExplicitUserItemBiases"]
residual_alphas = ["ExplicitUserItemBiases"] #
outdir = "NeuralImplicitItemCFUntuned"
tune_hyperparams = false

false

In [3]:
hyp = create_hyperparams(model, content, residual_alphas, input_alphas)
hyp = @set hyp.batch_size = 1024
G = hyp

Hyperparameters:
model                  => item_based_collaborative_filtering
content                => explicit
implicit               => false
input_data             => explicit
input_alphas           => ["ExplicitUserItemBiases"]
output_data            => allitems
batch_size             => 1024
user_sampling_scheme   => 0.0
learning_rate          => 0.001
optimizer_weight_decay => 1.0e-5
optimizer              => SGD
seed                   => 20220524
num_users              => 4563688
holdout                => NaN
temporal_holdout       => NaN
item_weight_decay      => 0.0
residual_alphas        => ["ExplicitUserItemBiases"]
temporal_weight_decay  => 0.38285452
user_weight_decay      => -1.0


In [4]:
function nlopt_loss(λ, grad)
    _, _, loss = train_model(
        create_hyperparams(hyp, convert.(Float32, λ));
        epochs_per_checkpoint = get_epochs_per_checkpoint(hyp.model),
        patience = 0,
    )
    @info "$λ $loss"
    loss
end


nlopt_loss (generic function with 1 method)

In [5]:
global G = hyp
opt = get_optimizer(G.optimizer, G.learning_rate, G.optimizer_weight_decay)
rng = Random.Xoshiro(G.seed)
Random.seed!(rand(rng, UInt64))

TaskLocalRNG()

In [6]:
CUDA.functional()

true

In [7]:
CUDA.default_rng()

CUDA.RNG(0x34c1c755, 0x00000000)

In [8]:

if CUDA.functional()
    Random.seed!(CUDA.default_rng(), rand(rng, UInt64))
    Random.seed!(CUDA.CURAND.default_rng(), rand(rng, UInt64))
end

In [7]:
# λ = convert.(Float32, zeros(Float32, num_tuneable_params(hyp.model)))
# train_model(
#         create_hyperparams(hyp, convert.(Float32, λ));
#         epochs_per_checkpoint = get_epochs_per_checkpoint(hyp.model),
#         patience = 0,
#     )

In [8]:
# nlopt_loss(zeros(Float32, num_tuneable_params(hyp.model)), 0)

In [9]:
# function nlopt_optimize(
#     lossfn,
#     n::Int;
#     max_evals::Int = 100,
#     max_time::Int = 86400,
#     ftol_rel::Real = 1e-4,
#     xtol_rel::Real = 1e-4,
# )
#     opt = NLopt.Opt(:LN_NELDERMEAD, n)
#     opt.initial_step = 1
#     opt.maxeval = max_evals
#     opt.maxtime = max_time
#     opt.ftol_rel = ftol_rel
#     opt.xtol_rel = xtol_rel
#     opt.min_objective = lossfn
#     minf, λ, ret = NLopt.optimize(opt, zeros(Float32, n))
#     numevals = opt.numevals
#     @info (
#         "found minimum $minf at point $λ after $numevals function calls " *
#         "(ended because $ret)"
#     )
#     convert.(Float32, λ)
# end;

#     nlopt_optimize(nlopt_loss, num_tuneable_params(hyp.model); max_evals = max_evals);

In [10]:
# train_alpha(
#     hyp,
#     outdir;
#     tune_hyperparams=tune_hyperparams
# )